In [1]:
from IPython.display import display, HTML

display(
    HTML(
        """<style>
* {font-family:D2Coding;}
div.container{width:87% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input { font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:3px;}
table.dataframe{font-size:12px;}
</style>
"""
    )
)

[ RAG 구현 절차 ]

1. 문서의 내용을 읽는다(document_loader를 이용)

- (1) https://python.langchain.com/v0.2/docs/integrations/document_loaders/
- (2) https://python.langchain.com/v0.2/docs/integrations/document_loaders/microsoft_word/
- %pip install --upgrade --quiet docx2txt

2. 문서를 쪼갠다(한번에 이해하고 처리할 수 있는 입력+출력 토큰수가 제한)

- (1) https://python.langchain.com/v0.2/docs/how_to/recursive_text_splitter/#splitting-text-from-languages-without-word-boundaries
- %pip install -qU langchain-text-splitters

3. 쪼갠 문서를 임베딩하여 vector database에 넣음

- (1) OpenAIEmbeddings나 UpstageEmbeddings이용해서 임베딩
- (2) https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/
- %pip install –q langchain-chroma

4. 질문을 이용해 유사도 검색
5. 유사도 검색한 문서를 LLM에 질문으로 전달하여 답변 얻음(제공되는 Prompt활용)

- (1) https://python.langchain.com/v0.2/docs/tutorials/rag/
- %pip install –q langchain langchainhub

- https://smith.langchain.com/ 에서 key 생성 .env key 추가


# 2. 문서를 쪼개면서 읽기 (O)


In [1]:
import time

start = time.time()

from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import (
    RecursiveCharacterTextSplitter,
)  # 문서를 읽어오면서 겹치게 발라줌

loader = Docx2txtLoader("./tax_docs/소득세법(법률)(제20615호)(20250701).docx")

# 문자단위로 쪼갬
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, chunk_overlap=200  # 문서를 쪼갤떄 1500글자씩 쪼개
)
# 1번째 chunk 1 ~ 1450글자
# 2번쨰 chunk 1250 ~ 1750글자

documents = loader.load_and_split(text_splitter=text_splitter)

runtime = time.time() - start
print("문서 쪼개면서 읽는 시간", runtime)

문서 쪼개면서 읽는 시간 10.986136198043823


# 3. 쪼갠문서를 임베딩 -> 벡터 데이터베이스 저장

- Embedding Model : Upstage solar-embedding-1-large, embedding-query
- 벡터 데이터베이스 : chroma


In [2]:
# 사용법 : https://python.langchain.com/v0.2/docs/integrations/text_embedding/upstage/
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings

load_dotenv()
embeddings = UpstageEmbeddings(
    model="solar-embedding-1-large",
    # model="embedding-query"
)

In [4]:
doc_result = embeddings.embed_documents(
    ["소득세법 어쩌구 저쩌구", documents[0].page_content]
)
print(len(doc_result), len(doc_result[0]), len(doc_result[1]))

2 4096 4096


In [ ]:
%%time
from langchain_chroma import Chroma

# 데이터를 처음 저잘할떄
# database = Chroma.from_documents(
#     documents=documents,
#     embedding=embeddings,
#     collection_name='tax_collection',# 생략시 이름 랜덤
#     persist_directory='./chroma_upstage',# 생략시 로컬데이터베이스에 저장안됨, 프로그램 종료시 db날라감
# )
# 계속 실행하면 하면 안됨. 꼭 한번만. 그래서 주석처리


CPU times: total: 15.6 s
Wall time: 55.8 s


In [7]:
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings
from langchain_chroma import Chroma
%time
load_dotenv()
embeddings = UpstageEmbeddings(
    model="solar-embedding-1-large",
    # model="embedding-query"
)

# 이미 저장된 vector DB를 사용할떄
database = Chroma(
    embedding_function=embeddings,
    collection_name='tax_collection',
    persist_directory='./chroma_upstage'
)

CPU times: total: 0 ns
Wall time: 0 ns


# 4. vector DB 에 질문과 유사도 검색 (답변 생성을 위한 retrieval)


In [ ]:
query = "연봉 5000만원인 직장인의 소득세는 얼마인가요?"

retrieved_docs = database.similarity_search(query, k=3)  # 기본 k값이 4

In [9]:
retrieved_docs

[Document(id='be52c9e4-4e47-4783-b4ae-6427de957f58', metadata={'source': './tax_docs/소득세법(법률)(제20615호)(20250701).docx'}, page_content='2. 2명인 경우: 연 55만원\n\n3. 3명 이상인 경우: 연 55만원과 2명을 초과하는 1명당 연 40만원을 합한 금액\n\n② 삭제<2017. 12. 19.>\n\n③ 해당 과세기간에 출산하거나 입양 신고한 공제대상자녀가 있는 경우 다음 각 호의 구분에 따른 금액을 종합소득산출세액에서 공제한다.<신설 2015. 5. 13., 2016. 12. 20.>\n\n1. 출산하거나 입양 신고한 공제대상자녀가 첫째인 경우: 연 30만원\n\n2. 출산하거나 입양 신고한 공제대상자녀가 둘째인 경우: 연 50만원\n\n3. 출산하거나 입양 신고한 공제대상자녀가 셋째 이상인 경우: 연 70만원\n\n④ 제1항 및 제3항에 따른 공제를 “자녀세액공제”라 한다.<신설 2015. 5. 13., 2017. 12. 19.>\n\n[본조신설 2014. 1. 1.]\n\n[종전 제59조의2는 제59조의5로 이동 <2014. 1. 1.>]\n\n\n\n제59조의3(연금계좌세액공제) ① 종합소득이 있는 거주자가 연금계좌에 납입한 금액 중 다음 각 호에 해당하는 금액을 제외한 금액(이하 “연금계좌 납입액”이라 한다)의 100분의 12[해당 과세기간에 종합소득과세표준을 계산할 때 합산하는 종합소득금액이 4천 500만원 이하(근로소득만 있는 경우에는 총급여액 5천 500만원 이하)인 거주자에 대해서는 100분의 15]에 해당하는 금액을 해당 과세기간의 종합소득산출세액에서 공제한다. 다만, 연금계좌 중 연금저축계좌에 납입한 금액이 연 600만원을 초과하는 경우에는 그 초과하는 금액은 없는 것으로 하고, 연금저축계좌에 납입한 금액 중 600만원 이내의 금액과 퇴직연금계좌에 납입한 금액을 합한 금액이 연 900만원을 초과하는 경우에는 그 초과하는 금액은 없는

# 5. 유사도 검색으로 가져온 문서를 질문과 같이 LLM 전달하여 답변 생성


In [10]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-nano")

In [11]:
prompt = f"""[identity]
- 당신은 최고의 한국 소득세 전문가입니다.
- [context]를 참고해서 사용자의 질문에 답변해 주세요.
[context]는 다음과 같습니다.
{retrieved_docs}
Question: {query}
"""

In [12]:
ai_message = llm.invoke(prompt)

In [13]:
print(ai_message.content)

연봉 5000만원인 직장인의 소득세를 계산하기 위해서는 먼저 근로소득공제와 표준세율을 적용하여 과세표준을 산출한 후, 해당 과세표준에 따른 세율을 적용해야 합니다. 아래는 일반적인 계산 방법입니다.

1. 근로소득공제 계산
- 연봉 5000만원의 경우, 근로소득공제는 2천만원이 최대 공제액이므로, 연봉이 5000만원일 때는 2천만원이 공제됩니다.
- 따라서 과세표준은 5000만원 - 2000만원 = 3000만원입니다.

2. 세율 적용
- 제55조에 따른 세율 표에 따라 3000만원의 과세표준에 해당하는 세율을 적용합니다.
- 2022년 기준 세율 구조는 대략 다음과 같습니다. (구체적 세율은 세법 개정에 따라 변동 가능, 참고용)

| 과세표준                            | 세율  | 누진공제액  |
|----------------------------------|-------|------------|
| 1,200만원 이하                   | 6%    | 0원        |
| 1,200만원 초과 ~ 4,600만원 이하 | 15%   | 108만원    |
| 4,600만원 초과                   | 24%   | 522만원    |

3. 과세표준 3000만원에 대한 세금 계산
- 1,200만원에 대해서는 6% 적용 → 1,200만원 × 0.06 = 72만원
- 나머지 1,800만원에 대해서는 15% 적용 → 1,800만원 × 0.15 = 270만원
- 전체 세액 = 72만원 + 270만원 = 342만원
- 세액에서 공제액(누진공제액)을 뺌 → 세금 = 342만원 - 108만원(누진공제액) = 234만원

※ 참고로, 주민세(지역소득세) 및 근로소득공제, 기타 공제액을 감안하면 최종 납부세액은 조금 차이날 수 있습니다.

결론적으로, 연봉 5000만원인 직장인의 예상 소득세는 약 234만원 정도입니다. 정확한 세액 계산을 위해서는 구체적인 공제내역과 연말정산 내용 등을 반영해야 하며, 세법 개정에 따

# 5. Agumentation 을 위한 제공하는 Prompt 활용하여, langchain으로 답변 생성

- 사용법 : https://python.langchain.com/v0.2/docs/tutorials/rag/


In [14]:
query = "연봉 5000만원인 직장인의 소득세는 얼마인가요?"

from langchain import hub

prompt = hub.pull("rlm/rag-prompt")
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

## RetrievalQA를 통해 LLM전달 (create_retrieval_chain이 대체)

```
 query -> retriever 전달(벡터 검색 수행)
 -> retriever문서 -> prompt의 {context}에 삽입
 -> query -> prompt의 (question)에 삽입
```


In [15]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=database.as_retriever(search_kwargs={"k": 5}),
    chain_type_kwargs={"prompt": prompt},
)

In [16]:
ai_message = qa_chain.invoke({"query": query})

In [17]:
ai_message

{'query': '연봉 5000만원인 직장인의 소득세는 얼마인가요?',
 'result': '연봉 5000만원인 직장인의 소득세는 약 835만원입니다. 이 금액은 근로소득공제, 자녀세액공제, 그리고 세율에 따라 계산된 결과입니다. 정확한 금액은 개인의 상세 상황에 따라 달라질 수 있습니다.'}